In [1]:
import csv,datetime,time
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import matplotlib.dates as mdate
import psycopg2
import sys

import pandas as pd

import plotting_functions as plt2


In [2]:
dsn  = "dbname=gm2_online_prod user=gm2_reader host=g2db-priv port=5433"
conn = psycopg2.connect(dsn)
curr = conn.cursor()


In [3]:
# choose data set

start_run = 15921  # 60 hr
end_run = 15992

In [4]:
# get times by subrun

subrun_time_columns = ['run', 'subrun', 'start_time', 'end_time', 'start_gps', 'end_gps']

sql = "select "+", ".join(subrun_time_columns)+" from gm2dq.subrun_time where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)

curr.execute(sql)
conn.commit()
subrun_time = curr.fetchall()

subrun_time_df = pd.DataFrame(subrun_time, columns=subrun_time_columns)

print subrun_time_df.shape

(27311, 6)


In [5]:
subrun_time_df.head()

,run,subrun,start_time,end_time,start_gps,end_gps
0,15921,0,2018-04-22 13:14:00,2018-04-22 13:14:34,2018-04-22 13:14:24.326720,2018-04-22 13:14:32.777180
1,15921,1,2018-04-22 13:14:36,2018-04-22 13:14:43,2018-04-22 13:14:32.787180,2018-04-22 13:14:41.414020
2,15921,2,2018-04-22 13:14:44,2018-04-22 13:14:52,2018-04-22 13:14:41.424020,2018-04-22 13:14:49.864100
3,15921,3,2018-04-22 13:14:53,2018-04-22 13:15:00,2018-04-22 13:14:50.927420,2018-04-22 13:14:59.377630
4,15921,4,2018-04-22 13:15:02,2018-04-22 13:15:19,2018-04-22 13:14:59.387640,2018-04-22 13:15:17.694490


In [5]:
# get ctags by subruns

ctagswithdqc_columns = ['run', 'subrun', 'ctags', 't0val', 'fills']

sql = "select "+", ".join(ctagswithdqc_columns)+" from gm2dq.ctagswithdqc where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)

curr.execute(sql)
conn.commit()
ctagswithdqc = curr.fetchall()

ctagswithdqc_df = pd.DataFrame(ctagswithdqc, columns=ctagswithdqc_columns)

print ctagswithdqc_df.shape

(27275, 5)


In [6]:
# get subrun status database

subrun_status_columns = ['run', 'subrun', 'quad_condition', 'kicker_condition', 'quad_ok',
                         'ctags_ok', 'losses_ok', 'fillcuts_ok', 'field_ok', 'trolley_period', 'field_period',
                         'ctags_loose_ok', 'quad_loose_ok', 'ctags_repeat_ok', 'losses_repeat_ok', 'fillcuts_repeat_ok']

sql = "select "+", ".join(subrun_status_columns)+" from gm2dq.subrun_status where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)

curr.execute(sql)
conn.commit()
subrun_status = curr.fetchall()

subrun_status_df = pd.DataFrame(subrun_status, columns=subrun_status_columns)

print subrun_status_df.shape

(27311, 16)


In [7]:
# merge times, ctags, status into one subrun dataframe

subrun_df = pd.merge(subrun_time_df, ctagswithdqc_df)
subrun_df = pd.merge(subrun_df, subrun_status_df)
subrun_df['ok'] = subrun_df['quad_ok'] & subrun_df['ctags_ok'] & subrun_df['losses_ok'] & subrun_df['fillcuts_ok'] & subrun_df['field_ok']

subrun_df['start_time'] = subrun_df['start_time'].astype(np.int64)/1e9 + 5*60*60
subrun_df['end_time'] = subrun_df['end_time'].astype(np.int64)/1e9 + 5*60*60
subrun_df['start_gps'] = subrun_df['start_gps'].astype(np.int64)/1.0e9 + 5*60*60
subrun_df['end_gps'] = subrun_df['end_gps'].astype(np.int64)/1.0e9 + 5*60*60

subrun_df.to_hdf('60hr_subrun.h5', key='ctag')

print subrun_df.shape

(27275, 24)


/home/aetb/.local/lib/python2.7/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['quad_condition', 'kicker_condition', 'field_ok', 'quad_loose_ok', 'ctags_repeat_ok', 'losses_repeat_ok', 'fillcuts_repeat_ok']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [ ]:
print subrun_df['start_gps'].iloc[16]
print subrun_df['start_time'].iloc[16]

In [ ]:
subrun_df[subrun_df['ok']==True].head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(times,ctags,marker="o",linewidth=0,color="green",markeredgewidth=0)
date_fmt = '%d-%b\n%H:%M'
date_formatter = mdate.DateFormatter(date_fmt)
hours = mdate.HourLocator(interval = 12)
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(date_formatter)
ax.set_ylabel("# CTAGs")
ax.set_title("CTags in 60Hr dataset")
fig.autofmt_xdate()
# plt.savefig("ctag_60Hr.png")

In [ ]:
# get database entries for ctags from DQM

start_time = "2018-04-22 00:00:00"
end_time   = "2018-04-22 23:59:59"

sql = "select * from gm2ctag_dqm where time >= '%s' and time <= '%s' order by time DESC" % (start_time, end_time)

curr.execute(sql)
conn.commit()
ctag_dqm = curr.fetchall()

ctag_dqm = np.array(ctag_dqm)

for row in ctag_dqm:
    row[0] = time.mktime(row[0].timetuple())
    
ctag_dqm = ctag_dqm[ctag_dqm[:,0].argsort()]

# plt.plot(ctag_dqm[:,0], ctag_dqm[:,1], '.')